In [ ]:
#Mapping Agricultural Cover in Rhode Island

# Import Libraries
import geopandas as gpd               # for handling .shp/geospatial data
import numpy as np                    # for numerical operations, reclass
import matplotlib.pyplot as plt        # for visualizing data
import matplotlib.colors as mcolors
import pandas as pd                   # for attribute tables/data

print("All libraries loaded successfully")

# Define file paths. Soil, Land use, and Town data can be downloaded from RIGIS
# These paths are really the only thing you need to edit (besides the town to zoom into)
land_use_path = "/Users/michaelc/Downloads/LandCoverSHP/PLAN_Land_Cover_Use_2020.shp"
towns_path = "/Users/michaelc/Downloads/Lab_1_Data/towns.shp"

# Load both shapefiles
land_use_gdf = gpd.read_file(land_use_path)
towns_gdf = gpd.read_file(towns_path)

# Reproject to EPSG:4326 (if not already in lat/long)
land_use_gdf = land_use_gdf.to_crs(epsg=4326)

# Print first few rows of attribute tables
print("Land Use Data:")
print(land_use_gdf.head())

# Print Unique descriptions for reclassifying agricultural cover
print("Unique Land Use Descriptions:")
print(land_use_gdf['Descr_2020'].unique())

# Reclassify land cover classes to create new values ranging from 0-3
# Flexibility in this step, can reclass attribute values differently to display different things. ex. impervious cover - imp. cover reclass is provided for example use. 
reclassification_dict = {
    'Cemeteries': 0,
    'Water': 0,
    'Vacant Land': 0,
    'Wetland': 0,
    'Idle Agriculture (abandoned fields and orchards)': 1,
    'Brushland (shrub and brush areas, reforestation)': 0,
    'Deciduous Forest (>80% hardwood)': 0,
    'Softwood Forest (>80% softwood)': 0,
    'Mixed Forest': 0,
    'Pasture (agricultural not suitable for tillage)': 1,
    'Cropland (tillable)': 3,
    'Orchards, Groves, Nurseries': 2,
    'Sandy Areas (not beaches)': 0,
    'Rock Outcrops': 0,
    'Mines, Quarries and Gravel Pits': 0,
    'Institutional (schools, hospitals, churches, etc.)': 0,
    'Transitional Areas (urban open)': 0,
    'Mixed Barren Areas': 0,
    'Low Density Residential (>2 acre lots)': 0,
    'Medium Low Density Residential (1 to 2 acre lots)': 0,
    'Medium Density Residential (1 to 1/4 acre lots)': 0,
    'Power Lines (100\' or more width)': 0,
    'Commercial/Residential Mixed': 0,
    'Developed Recreation (all recreation)': 0,
    'Medium High Density Residential (1/4 to 1/8 acre lots)': 0,
    'High Density Residential (<1/8 acre lots)': 0,
    'Commercial (sale of products and services)': 0,
    'Industrial (manufacturing, design, assembly, etc.)': 0,
    'Roads (divided highways >200\' plus related facilities)': 0,
    'Airports (and associated facilities)': 0,
    'Railroads (and associated facilities)': 0,
    'Water and Sewage Treatment': 0,
    'Waste Disposal (landfills, junkyards, etc.)': 0,
    'Ground-mounted Solar Energy Systems': 0,
    'Beaches': 0,
    'Wind Energy Systems': 0,
    'Commercial/Industrial Mixed': 0,
    'Confined Feeding Operations': 1,
    'Other Transportation (terminals, docks, etc.)': 0
}

#reclassification_dict = {
    #'Cemeteries': 0,
    #'Water': 0,
    #'Vacant Land': 0,
    #'Wetland': 0,
    #'Idle Agriculture (abandoned fields and orchards)': 0,
    #'Brushland (shrub and brush areas, reforestation)': 0,
    #'Deciduous Forest (>80% hardwood)': 0,
    #'Softwood Forest (>80% softwood)': 0,
    #'Mixed Forest': 0,
    #'Pasture (agricultural not suitable for tillage)': 0,
    #'Cropland (tillable)': 0,
    #'Orchards, Groves, Nurseries': 0,
    #'Sandy Areas (not beaches)': 0,
    #'Rock Outcrops': 0,
    #'Mines, Quarries and Gravel Pits': 0,
    #'Institutional (schools, hospitals, churches, etc.)': 1,
    #'Transitional Areas (urban open)': 1,
    #'Mixed Barren Areas': 0,
    #'Low Density Residential (>2 acre lots)': 1,
    #'Medium Low Density Residential (1 to 2 acre lots)': 1,
    #'Medium Density Residential (1 to 1/4 acre lots)': 2,
    #'Power Lines (100\' or more width)': 2,
    #'Commercial/Residential Mixed': 2,
    #'Developed Recreation (all recreation)': 2,
    #'Medium High Density Residential (1/4 to 1/8 acre lots)': 3,
    #'High Density Residential (<1/8 acre lots)': 3,
    #'Commercial (sale of products and services)': 3,
    #'Industrial (manufacturing, design, assembly, etc.)': 3,
    #'Roads (divided highways >200\' plus related facilities)': 3,
    #'Airports (and associated facilities)': 3,
    #'Railroads (and associated facilities)': 3,
    #'Water and Sewage Treatment': 2,
    #'Waste Disposal (landfills, junkyards, etc.)': 2,
    #'Ground-mounted Solar Energy Systems': 3,
    #'Beaches': 0,
    #'Wind Energy Systems': 3,
    #'Commercial/Industrial Mixed': 3,
    #'Confined Feeding Operations': 3,
    #'Other Transportation (terminals, docks, etc.)': 3
}

#------------------------------------------------------------------------------

# Create a new reclassified column based on agricultural cover focus
land_use_gdf['Reclassified'] = land_use_gdf['Descr_2020'].map(reclassification_dict)

# Check the reclass by printing
print(land_use_gdf[['Descr_2020', 'Reclassified']].drop_duplicates())

# Calculate area in square meters
#town_land_use['area'] = town_land_use.geometry.area
 
    # Ensure towns_gdf is in the same CRS as land_use_gdf
towns_gdf = towns_gdf.to_crs(land_use_gdf.crs)

# Custom colormap with agricultural focus
cmap = mcolors.ListedColormap(['gray', 'yellow', 'orange', 'green'])
bounds = [0, 1, 2, 3, 4] 
norm = mcolors.BoundaryNorm(bounds, cmap.N)

# Create a figure with large size for mapping
fig, ax = plt.subplots(figsize=(50, 50))

# Plot land use data using reclassified values and colormap
land_use_gdf.plot(column='Reclassified', ax=ax, cmap=cmap, norm=norm)

# Overlay the town boundaries on top of the land use map
towns_gdf.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=2)

# Create a colorbar with specific labels
cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax, orientation='horizontal')
cbar.set_label('Land Use Classification')
cbar.set_ticks([0, 1, 2, 3])
cbar.set_ticklabels(['Not Agricultural (0)', 'Idle Agriculture and Feeding Operations (1)', 'Tillable Cropland and Orchards, Groves, and Nurseries (2)', 'Null (3)']) 
# Set title and labels
ax.set_title('Agricultural Cover in Rhode Island', fontsize=50)
ax.set_xlabel('Longitude', fontsize=50)
ax.set_ylabel('Latitude', fontsize=50)

# Tick label size
ax.tick_params(axis='both', labelsize=20)

# Show plot
plt.show()

#-----------------------------------------------------------------------

#ZOom to town and display stats

def zoom_to_town_and_display_agricultural_cover(town_name, land_use_gdf, towns_gdf, cmap, norm):
    """
    Zoom into the specific town, plot the land use data with town boundaries, 
    and display agricultural cover statistics for the town.
    
    Parameters:
    town_name (str): The name of the town to zoom into.
    land_use_gdf (GeoDataFrame): The GeoDataFrame containing the land use data.
    towns_gdf (GeoDataFrame): The GeoDataFrame containing the town boundaries.
    cmap (ListedColormap): The colormap for land use classification.
    norm (BoundaryNorm): The normalization for colormap.
    """
    
    # Ensure towns_gdf is in the same CRS as land_use_gdf
    towns_gdf = towns_gdf.to_crs(land_use_gdf.crs)

    # Get the specific town from the towns_gdf
    town = towns_gdf[towns_gdf['NAME'] == town_name]

    # Check if the town exists
    if town.empty:
        print(f"Town '{town_name}' not found!")
        return

    # Get the bounding box (extent) of the town
    minx, miny, maxx, maxy = town.geometry.total_bounds

    # Clip the land use data to the town boundary
    town_land_use = land_use_gdf.clip(town)

    # Create a new figure for the zoomed-in view
    fig, ax = plt.subplots(figsize=(15, 15))

    # Plot the land use data
    town_land_use.plot(column='Reclassified', ax=ax, cmap=cmap, norm=norm)

    # Overlay the town boundaries
    town.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=2)

    # Set the axis limits to zoom into the selected town
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    # Create a colorbar with specific labels
    cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax, orientation='horizontal')
    cbar.set_label('Agricultural Use Classification')
    cbar.set_ticks([0, 1, 2, 3])  # Set ticks for the colorbar
    cbar.set_ticklabels(['Not Agricultural (0)', 'Idle Agriculture and Feeding Operations (1)', 'Orchards, Groves, and Nurseries (2)', 'Cropland (tillable) (3)'])  # Set tick labels

    # Set title and labels
    ax.set_title(f'Agricultural Cover in {town_name}', fontsize=20)
    ax.set_xlabel('Longitude', fontsize=15)
    ax.set_ylabel('Latitude', fontsize=15)

     # Tick label size
    ax.tick_params(axis='both', labelsize=12)
     # Show the plot
    plt.show()

#-----------------------------------------------------------------
# Now calculate the impervious cover statistics for the town

    # Reproject to a projected CRS (e.g., UTM Zone 19N for Rhode Island)
    town_land_use = town_land_use.to_crs(epsg=32619)

    # Add a calculated 'area' column if it doesn't exist
    if 'area' not in town_land_use.columns:
        town_land_use['area'] = town_land_use.geometry.area

    # Total area of the town
    total_area = town_land_use['area'].sum()
    print(f"Total Area for {town_name}: {total_area} square meters")

    # Filter for impervious cover categories (e.g., 1, 2, 3)
    agricultural_land_use = town_land_use[town_land_use['Reclassified'].isin([1, 2, 3])]

    # Impervious area
    agricultural_area = agricultural_land_use['area'].sum()
    print(f"Total Agricultural Area for {town_name}: {agricultural_area} square meters")

    # Calculate impervious cover percentage
    agricultural_percentage = (agricultural_area / total_area) * 100
    print(f"Agricultural Cover Percentage for {town_name}: {agricultural_percentage}%")

    # Breakdown by land use category (impervious categories 1, 2, 3)
    agricultural_by_category = agricultural_land_use.groupby('Reclassified')['area'].sum()

    # Create a summary DataFrame
    agricultural_stats = pd.DataFrame({
        'Land Use Category': agricultural_by_category.index,
        'Area (sq meters)': agricultural_by_category.values,
        'Percentage of Total Agricultural Area': (agricultural_by_category / agricultural_area) * 100
    })

    # Add total impervious cover statistics
    agricultural_stats.loc['Total'] = [
        'Total Agricultural Cover',
        agricultural_area,
        agricultural_percentage
    ]

    # Print the impervious cover statistics
    print(f"agricultural Cover Stats for {town_name}:")
    print(agricultural_stats)

# Example usage: Zoom into a specific town and display impervious cover stats
town_name = "CRANSTON"  # Replace with the Rhode Island town you want to zoom into
zoom_to_town_and_display_agricultural_cover(town_name, land_use_gdf, towns_gdf, cmap, norm)